In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from threeML import *
import numpy as np
from threeML.plugins import *
from skylab.datasets import Datasets
from astropy import units as u

import os, sys, glob, abc

def read(filelist):
    data = []
    for f in sorted(filelist):
        x = np.load(f)
        if len(data) == 0: data = x.copy()
        else: data = np.concatenate([data, x])
    return data

# Where is the dataset stored?
dataset = "/data/i3store/users/mjlarson/student_data"

# Read in all of the data files
data_files = dataset + "/IC86_*exp.npy"
exp = read(glob.glob(data_files))

# Read in all of the MC files 
sim_files = dataset + "/IC86*MC*npy"

mc = read(glob.glob(sim_files))
sample, season = 'GFU', 'IC86, 2011-2018'
#sample, season = 'PointSourceTracks', 'IC86, 2012-2017'
d = Datasets[sample]
d.set_repository_path("/data/i3store/users/mjlarson/")
#data, mc, livetime = d.season(season, compress=False)
#grl = d.grl(season)
sinDec_bins = d.sinDec_bins(season)
energy_bins = d.energy_bins(season)
dataset = "/data/i3store/users/mjlarson/student_data"

# Read in all of the data files
data_files = dataset + "/IC86_*exp.npy"
exp = read(glob.glob(data_files))

# Read in all of the MC files 
sim_files = dataset + "/IC86*MC*npy"
mc = read(glob.glob(sim_files))

# Set the angular error floor to 0.2 degrees
#exp['angErr'][exp['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
#mc['angErr'][mc['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)

exp['ra']=np.random.uniform(0,2*np.pi,size=len(exp))
grl = np.load("/data/i3store/users/mjlarson/student_data/GRL/IC86_2012_exp.npy")
livetime = np.sum(grl['livetime'])




Bad key "text.kerning_factor" on line 4 in
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
/data/disk01/home/jasonfan/threeML/threeML/__init__.py:12: UserWarning: No DISPLAY variable set. Using backend for graphics without display (Agg)
  warnings.warn("No DISPLAY variable set. Using backend for graphics without display (Agg)")
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/astromodels/core/parameter.py:555: UserWarning: We have set the min_value of K to 1e-99 because there was a postive transform
  warnings.warn('We have set the min_value of %s to 1e-99 because there was a postive transform' % self.path)
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/astromodels/core/paramet

In [2]:
from imp import reload
reload(IceCubeLike)

import warnings
warnings.filterwarnings("ignore")
#data = np.random.choice(exp, 10000).copy()
#jl._data_list['test'].llh_model.update_data(data)
IceCube=IceCubeLike.IceCubeLike("test",exp,mc,exp,verbose=True,background_time_profile= (56224,56324),sampling_width=np.radians(4),bkg_2dbins=[sinDec_bins,energy_bins])
del mc

In [3]:
source1_sp = Powerlaw()
#source1_sp._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
source1_sp.K=1e-13
source1_neutrino = Powerlaw()
source1_neutrino._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
source1_neutrino.piv=1000
source1_neutrino.K=1e-13
source1_neutrino.index=-2
#source1_sp._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
source1 = PointSource("source1", ra=83.63, dec=22.0145, spectral_shape=source1_sp)
source1_nu = IceCubeLike.NeutrinoPointSource("source1_nu",ra=83.63, dec=45,spectral_shape=source1_neutrino,energy_unit=u.GeV)

model = Model(source1,source1_nu)
model.source1.spectrum.main.Powerlaw.K.fix = True
model.source1.spectrum.main.Powerlaw.index.fix = True
model.source1_nu.spectrum.main.Powerlaw.K.bounds = (1e-30, 1e-5)
model.source1_nu.spectrum.main.Powerlaw.index.bounds = (-4, -1)
#model.source1_nu.spectrum.main.Powerlaw.index.fix = True
#IceCube.set_model(model)
IceCubedata = DataList(IceCube)
jl = JointLikelihood(model, IceCubedata)

In [4]:
test=IceCubeLike.sensitivity(jl)

In [ ]:
test.set_backround(grl,model ,time_window=100,start_time=56224)

In [ ]:
import mla 
basespectrum = mla.spectral.PowerLaw(1e3 , 1 , -2)
test.set_injection(background_time_profile = (56224,56324))

In [ ]:
sp=mla.spectral.PowerLaw(1e3 , 1e-9 , -2)
injector=mla.injection.PSinjector(sp,test.jl_value.llh_model.fullsim,background_time_profile = (56224,56324))
injector.set_backround(exp,grl,background_window=100)
injector.set_source_location(np.radians(83.63) ,np.radians(45) )


In [ ]:
exp

array([(120028,  839634, 0, 3.37669699, -0.99286837, 3.35256417, 0.57782915, 56043.42668248, 4.94667356, 0.00279098),
       (120028, 1578865, 0, 1.52056846,  0.02396377, 5.00543175, 1.59358036, 56043.42980643, 3.01017533, 0.00650307),
       (120028, 2116382, 0, 0.64200876,  0.15281481, 5.28118174, 1.72253646, 56043.43207568, 3.57020037, 0.01181358),
       ...,
       (122275, 8273253, 0, 0.42437671,  0.01545653, 5.86173696, 1.58559894, 56414.40951669, 3.48505343, 0.0047242 ),
       (122275, 8753212, 0, 5.09361497,  1.10413968, 2.97424766, 2.67519436, 56414.41156909, 3.58320322, 0.01279018),
       (122275, 9731200, 0, 4.39993599, -0.86680749, 1.72704938, 0.70527777, 56414.41574579, 4.65537209, 0.0035007 )],
      dtype=[('run', '<i8'), ('event', '<i8'), ('subevent', '<i8'), ('ra', '<f8'), ('dec', '<f8'), ('azi', '<f8'), ('zen', '<f8'), ('time', '<f8'), ('logE', '<f8'), ('angErr', '<f8')])

In [ ]:
test.jl_value.llh_model.update_data(injector.draw_data())
test.jl_value.llh_model.modify_injection(injector.sample_nevents(15))
test.jl_value.verbose=True
test.jl.fit()

(37.300933866391, 18.01561168338722)
(37.52493121174089, 17.807595961230867)
(37.07825452589248, 18.221489105480742)
(37.3232741644608, 17.994906574755177)
(37.278606748334155, 18.03629540909374)
(37.3461551380384, 17.955319456064863)
(37.25578260847507, 18.075765023554546)
(37.31232886119503, 18.00042336033856)
(37.28954332229603, 18.030791177502856)
(12.84658361757328, 35.45613903795292)
(15.73288273022545, 35.10252057261107)
(13.54784643527122, 35.44824408851326)
(12.853528795257677, 35.457811312160935)
(12.839642142535487, 35.454462534257374)
(12.860570741387953, 35.459501658214066)
(12.832611503309106, 35.45275927245072)
(12.851171577312183, 35.45497706270923)
(12.841998061736863, 35.45729720650863)
(12.856323310460226, 35.45366849106756)
(12.836854751800942, 35.45859243962005)
(13.148629750146624, 35.688211187486004)
(14.47123681271168, 36.51054811897295)
(18.77277027499892, 37.58767905347566)
(20.24496796943547, 37.55348124212596)
(18.79359225112394, 37.58451436803162)
(18.75197

,result,unit
parameter,,
source1_nu.spectrum.main.Powerlaw.K,(1.5 -0.4 +0.6) x 10^-19,1 / (cm2 keV s)
source1_nu.spectrum.main.Powerlaw.index,-2.52 +/- 0.17,



Correlation matrix:



1.00,-0.76
-0.76,1.00



Values of -log(likelihood) at the minimum:



,-log(likelihood)
test,-38.472753
total,-38.472753



Values of statistical measures:



,statistical measures
AIC,-78.945506
BIC,-76.945506


(                                                value  negative_error  \
 source1_nu.spectrum.main.Powerlaw.K      1.451017e-19    1.026674e-13   
 source1_nu.spectrum.main.Powerlaw.index -2.522644e+00   -1.649494e-01   
 
                                          positive_error         error  \
 source1_nu.spectrum.main.Powerlaw.K        2.012619e-13  1.519647e-13   
 source1_nu.spectrum.main.Powerlaw.index    1.677810e-01  1.663652e-01   
 
                                                     unit  
 source1_nu.spectrum.main.Powerlaw.K      1 / (cm2 keV s)  
 source1_nu.spectrum.main.Powerlaw.index                   ,
        -log(likelihood)
 test         -38.472753
 total        -38.472753)

In [ ]:
in_ns=np.arange(5,101)
def injectns(inject_ns):
    result = []
    for i in range(100):
        print(i)
        test.reset_model()
        test.jl_value.llh_model.update_data(injector.draw_data())
        test.jl_value.llh_model.modify_injection(injector.sample_nevents(inject_ns))
        test.jl_value.verbose=False
        test.jl.fit(quiet = True)
        index = test.jl.results.optimized_model._point_sources['source1_nu'].spectrum.main.Powerlaw.index
        ns = test.jl_value.llh_model.eval_llh()[0]
        result.append([inject_ns,index,ns])
    return np.array(result)

In [ ]:
temp=injectns(int(30))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [16]:
test.jl_value.llh_model.energybins

[array([-1.        , -0.9825    , -0.965     , -0.9475    , -0.93      ,
        -0.867     , -0.804     , -0.741     , -0.678     , -0.615     ,
        -0.552     , -0.489     , -0.426     , -0.363     , -0.3       ,
        -0.26111111, -0.22222222, -0.18333333, -0.14444444, -0.10555556,
        -0.06666667, -0.02777778,  0.01111111,  0.05      ,  0.10277778,
         0.15555556,  0.20833333,  0.26111111,  0.31388889,  0.36666667,
         0.41944444,  0.47222222,  0.525     ,  0.57777778,  0.63055556,
         0.68333333,  0.73611111,  0.78888889,  0.84166667,  0.89444444,
         0.94722222,  1.        ]),
 array([1.   , 1.125, 1.25 , 1.375, 1.5  , 1.625, 1.75 , 1.875, 2.   ,
        2.125, 2.25 , 2.375, 2.5  , 2.625, 2.75 , 2.875, 3.   , 3.125,
        3.25 , 3.375, 3.5  , 3.625, 3.75 , 3.875, 4.   , 4.125, 4.25 ,
        4.375, 4.5  , 4.625, 4.75 , 4.875, 5.   , 5.125, 5.25 , 5.375,
        5.5  , 5.625, 5.75 , 5.875, 6.   , 6.125, 6.25 , 6.375, 6.5  ,
        6.625, 6.75 , 6.8

In [13]:
result=[]
for i in temp[:,1]:
    result.append(i.value)

In [15]:
test

0.11715260731751798

In [ ]:
K=np.linspace(5e-14,1e-12,50)
gamma=np.linspace(-1,-4,30)
ns=np.linspace(0,60,50)
result=[]
for j in gamma:
    source1_neutrino = Powerlaw()
    source1_neutrino._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
    source1_neutrino.piv=1000
    source1_neutrino.index=j
    source1_nu = IceCubeLike.NeutrinoPointSource("source1_nu",ra=83.63, dec=22.0145,spectral_shape=source1_neutrino)
    model = Model(source1_nu)
    test.jl_value.set_model(model)
    test.jl_value.update_model()
    for i in K:
        source1_neutrino.K=i
        ns = (test.jl_value.llh_model.sim['ow'] * source1_neutrino(test.jl_value.llh_model.sim['trueE']) * test.jl_value.llh_model.signal_time_profile.effective_exposure() *24*3600).sum() 
        temp=test.jl_value.llh_model.eval_llh_ns(ns)
        result.append([i,j,temp[0],temp[1]])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
result=np.array(result)
x=result[:,0]
y=result[:,1]
z=result[:,3]

## Equivalently, we could do that all in one line with:
# x,y,z = np.genfromtxt('eye_.txt', delimiter=',', usecols=(0,1,2))
z[z<0]=0
x=np.unique(x)
y=np.unique(y)
X,Y = np.meshgrid(x,y)

Z=z.reshape(len(y),len(x))
index=result[np.argmax(result[:,3])]
fig, ax = plt.subplots(figsize = (6,5))
plt.pcolormesh(X,np.flip(Y),Z)
plt.colorbar()
plt.scatter(index[0],index[1],label="Best fit")
plt.title("Spectral index vs Flux Norm",fontsize=14)
plt.xlabel(r"Flux Norm($GeV^{-1} s^{-1} cm^{-2}$)",fontsize=14)
#plt.xlabel(r"ns",fontsize=14)
plt.ylabel("Spectral index",fontsize=14)
plt.legend()
plt.show()

In [ ]:
K=np.linspace(5e-14,1e-12,50)
gamma=np.linspace(-1,-4,30)
ns=np.linspace(0,70,100)
result=[]
for j in gamma:
    source1_neutrino = Powerlaw()
    source1_neutrino._set_units(u.GeV,u.cm**-2 / u.s / u.GeV)
    source1_neutrino.piv=1000
    source1_neutrino.index=j
    source1_nu = IceCubeLike.NeutrinoPointSource("source1_nu",ra=83.63, dec=22.0145,spectral_shape=source1_neutrino)
    model = Model(source1_nu)
    test.jl_value.set_model(model)
    test.jl_value.update_model()
    for i in ns:
        
        n_s = i 
        temp=test.jl_value.llh_model.eval_llh_ns(n_s)
        result.append([i,j,temp[0],temp[1]])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
result=np.array(result)
x=result[:,0]
y=result[:,1]
z=result[:,3]

## Equivalently, we could do that all in one line with:
# x,y,z = np.genfromtxt('eye_.txt', delimiter=',', usecols=(0,1,2))
z[z<0]=0
x=np.unique(x)
y=np.unique(y)
X,Y = np.meshgrid(x,y)

Z=z.reshape(len(y),len(x))
index=result[np.argmax(result[:,3])]
fig, ax = plt.subplots(figsize = (6,5))
plt.pcolormesh(X,np.flip(Y),Z)
plt.colorbar()
plt.scatter(index[0],index[1],label="Best fit")
plt.title("Spectral index vs Flux Norm",fontsize=14)
plt.xlabel(r"ns",fontsize=14)
#plt.xlabel(r"ns",fontsize=14)
plt.ylabel("Spectral index",fontsize=14)
plt.legend()
plt.show()

In [ ]:
from imp import reload
import scipy.optimize
reload(mla.core)

sim = read(glob.glob(sim_files))
import warnings
warnings.filterwarnings("ignore")
testmla=mla.core.LLH_point_source(np.radians(83.63) ,np.radians(22.0145) , testdata , sim, "PowerLaw", background = exp , \
                              background_time_profile = (56224,56324))

In [ ]:
#testmla.update_energy_weight()
testmla.get_fit_result()

In [ ]:
gamma=np.linspace(-1.5,-3.5,30)
ns=np.linspace(0,70,60)
result=[]
tempresult=np.zeros((60,30))
for i,ga in enumerate(gamma):
    testmla.update_energy_weight(gamma=ga)
    for j,nss in enumerate(ns):
        temp=testmla.eval_llh_ns(nss)
        result.append([ga,nss,temp[0],temp[1]])
        tempresult[j][i]=temp[1]

In [ ]:
#ns=testmla.get_fit_result()[1]
sob_ratios = testmla.evaluate_interpolated_ratio()
sob_spline = np.zeros(len(testmla.data), dtype=object)
for i in range(len(testmla.data)):
    spline = scipy.interpolate.UnivariateSpline(testmla.gamma_point,
                            np.log(sob_ratios[i]),
                            k = 3,
                            s = 0,
                            ext = 'raise')
    sob_spline[i] = spline

In [ ]:
def inner_ts(params):
    gamma=params[0]
    ns=params[1]
    e_lh_ratio = testmla.get_energy_sob(gamma, sob_spline)
    ts = ( ns/testmla.N * (e_lh_ratio*testmla.spatial*testmla.t_lh_ratio - 1))+1
    return -2*(np.sum(np.log(ts))+testmla.drop*np.log(1-ns/testmla.N))

bounds= [[testmla.gamma_point[0], testmla.gamma_point[-1]],[0,testmla.N]]
bf_params = scipy.optimize.minimize(inner_ts,
                    x0 = [-2,1],
                    bounds = bounds,
                    method = 'SLSQP',
                    )

In [ ]:
bf_params

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
result=np.array(result)
x=result[:,0]
y=result[:,1]
z=result[:,3]
gamma=np.linspace(-1.5,-3.5,30)
ns=np.linspace(0,70,60)
## Equivalently, we could do that all in one line with:
# x,y,z = np.genfromtxt('eye_.txt', delimiter=',', usecols=(0,1,2))
z[z<0]=0
x=np.unique(x)
y=np.unique(y)
X,Y = np.meshgrid(x,y)

Z=z.reshape(len(y),len(x))
#index=result[np.argmax(result[:,3])]
index=np.unravel_index(tempresult.argmax(), tempresult.shape)
fig, ax = plt.subplots(figsize = (6,5))
plt.pcolormesh(np.flip(X),Y,tempresult)
plt.colorbar()
plt.scatter(gamma[index[1]],ns[index[0]],label="Best fit")
plt.scatter(testmla.get_fit_result()[0],testmla.get_fit_result()[1],label="Best fit(MLA)")
plt.title("ns vs Spectral index",fontsize=14)
plt.xlabel(r"Spectral index",fontsize=14)
plt.ylabel("ns",fontsize=14)
plt.legend()
plt.show()

In [ ]:
gamma=np.linspace(-1.5,-3.5,30)
ns=np.linspace(0,70,60)
print(gamma[index[1]],ns[index[0]])